# <center> Module 3b - Basic Server
## <center> SYSE 549: Secure Vehicle and Industrial Networking
## <center> <img src="https://www.engr.colostate.edu/~jdaily/Systems-EN-CSU-1-C357.svg" width="400" /> 
### <center> Instructor: Dr. Jeremy Daily<br>Written By: Jerry Duggan

This notebook should be run along side the [BasicClient](./02b%20BasicClient.ipynb) notebook.

A **socket** is 
1. The point where application process attaches to the network.
2. An interface between an application and the network stack.
3. Sockets are created by local applications.

Step 1 - Server setup

In [1]:
import socket
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
print(s)

<socket.socket fd=1688, family=2, type=1, proto=0>


The variable `s` is a brand new socket.  Note that the fd (file descriptor) is set, and we know what kind of socket it is (AF_INET & SOCK_STREAM).  We have not specified any addressing information for it yet...

In [2]:
#help(socket.AddressFamily)

In [3]:
#help(socket.SocketKind)

Step 2 - Bind server socket to IP address & port

In [4]:
HOST = '127.0.0.1'  # Standard loopback interface address (localhost)
PORT = 12354        # Port to listen on (non-privileged ports are > 1023)

s.bind((HOST, PORT))
print(s)

<socket.socket fd=1688, family=2, type=1, proto=0, laddr=('127.0.0.1', 12354)>


We are binding the socket to a particular IP address & port.  Note that s now has the 'laddr' member set.  The first number the IP address on which the socket is opened, and the second is the port number for the connection. Since this is the local address parameter, and running in the server, the port in laddr matches the PORT parameter.

We have not told the OS whether this is going to be a "server" or a "client" socket.  The difference is that "server" sockets can accept connections while "client" sockets would make connections.

Step 3 - Server tell the OS that the application will be accepting connections on the socket

In [5]:
s.listen()
print(s)

<socket.socket fd=1688, family=2, type=1, proto=0, laddr=('127.0.0.1', 12354)>


No changes to the members of the socket at this step.

Make a note of the 'fd' (file descriptor) parameter in the socket

Step 4 - Server wait for connection

In [6]:
conn, addr = s.accept()
print(f"connection: {conn}")
print(f"connected from {addr}")

connection: <socket.socket fd=1936, family=2, type=1, proto=0, laddr=('127.0.0.1', 12354), raddr=('127.0.0.1', 51976)>
connected from ('127.0.0.1', 51976)


*The server accept() call is blocked, waiting for a client connection.*

**At this point, run Steps 5, 6, 7, and 8 in the Client notebook**

*The connection is now fully bound.  The 'laddr' parameter is the same as the socket, and it now has a 'raddr' (remote address) parameter that matches the host IP address and port from the client connection.*

*Note the 'fd' parameter in the connection -- it is different than the fd parameter in the socket.*

Step 9 - Server wait for input

In [7]:
data = conn.recv(1024)  # block until data is available
print(f"received data: {data}")

received data: b'Hello, Wireshark. You are an amazing tool!'


Step 10 - Server echo data back to client

In [8]:
ret = conn.sendall(data)

**If you want to send more data back and forth, redo steps 9 & 10 in the Server notebook and steps 7 & 8 in the Client notebook.**

**Try them in different orders (e.g. 9, then 7 & 8, then 10) and observe when blocking occurs.**

Step 11 - Server close connection

In [9]:
conn.close()
conn

<socket.socket [closed] fd=-1, family=2, type=1, proto=0>

At this point, go back and execute Steps 4, 9 and 10 in the server notebook & start from the beginning (step 5) in the Client notebook.  This will reuse the socket on the server side to create a new connection, and redo the complete creation process on the client side.

Step 13 -- Server Close Socket

In [10]:
s.close()
print(s)

<socket.socket [closed] fd=-1, family=2, type=1, proto=0>


**Now try to execute Steps 5 & 6 in the Client notebook.  This should throw a Connection Refused exception, as there is no application listening on port 12354 on the server.**

## IPv6
Try creating the socket with an IPv6 option.


In [ ]:
#help(socket.AddressFamily)

In [ ]:
s = socket.socket(socket.AF_INET6, socket.SOCK_STREAM)
print(s)

In [ ]:
HOST = '::1'  # Standard loopback interface address (localhost)
PORT = 12354        # Port to listen on (non-privileged ports are > 1023)

s.bind((HOST, PORT))
print(s)

## IPv6 Address Constants

IPv6 brings in a broader range of addresses, many of which serve similar roles as their IPv4 counterparts, but they also add entirely new types of addresses.

#### Unspecified Address: `::` or `in6addr_any`

Similar to INADDR_ANY, binding to the unspecified address means the socket accepts connections on all available IPv6 interfaces.

#### Loopback Address: `::1`  or  `in6addr_loopback`

Functions like INADDR_LOOPBACK in IPv4 because it restricts traffic to the local host. This is useful for testing local services without exposing them to an external network or creating some internal process communications.

#### Multicast Addresses:

IPv6 defines several multicast addresses (for instance, addresses for all nodes on the link-local network) that are used for group communication.

These are crucial for routing protocols and network discovery but are not typically used for binding listening sockets.

#### Link-local Addresses:

Start with `fe80::` and are automatically assigned to interfaces for communication on the local network segment.
